In [1]:
from network import *
from transformer import *

data, tokenizer = loadTransformerData('data/', singular=True)
vocabSize = len(tokenizer.wordMap)
print("Vocabulary size:", vocabSize)

p = data.take(1)
for inp, labels in p:
    x, context = inp
    for i in range(len(context.numpy())):
        print(x.numpy()[i])
        print(context.numpy()[i])
        print(labels.numpy()[i])
        print()

Tweets: 1117 Replies: 1114
Vocabulary size: 9167
[   1 5840   52 5841   21  843  916  579 5842    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
[   1    4  328 4186 8862    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
[   4  328 4186 8862    2    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]

[  1  35  87 706  10 707 397 460 243 203  59 70

In [5]:
data, tokenizers = loadStableData(1)

start_end = tokenizers.en.tokenize([''])[0]

start = start_end[0][tf.newaxis]
end = start_end[1][tf.newaxis]

output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
output_array = output_array.write(0, start)

output = tf.transpose(output_array.stack())

print(start, end, output_array, output)

# p = data.take(1)
# for inp, labels in p:
#     x, context = inp
#     for i in range(len(context.numpy())):
#         print(x.numpy()[i])
#         print(context.numpy()[i])
#         print(labels.numpy()[i])
#         print()

tf.Tensor([2], shape=(1,), dtype=int64) tf.Tensor([3], shape=(1,), dtype=int64) <tensorflow.python.ops.tensor_array_ops.TensorArray object at 0x0000024C004D4280> tf.Tensor([[2]], shape=(1, 1), dtype=int64)


In [2]:
numLayers = 4
embeddingDepth = 128
feedDepth = 512
numHeads = 8

model = Transformer(numLayers=numLayers, vocabSize=vocabSize, embeddingDepth=embeddingDepth, 
                    feedDepth=feedDepth, numHeads=numHeads)
model.compile(loss=masked_loss, optimizer='adam', metrics=[masked_accuracy])

In [3]:
model.fit(data, epochs=10)

Epoch 1/10
35/35 [==============================] - 158s 4s/step - loss: 7.9045 - masked_accuracy: 0.0696
Epoch 2/10
35/35 [==============================] - 135s 4s/step - loss: 6.8990 - masked_accuracy: 0.0718
Epoch 3/10
35/35 [==============================] - 142s 4s/step - loss: 6.8619 - masked_accuracy: 0.0719
Epoch 4/10
35/35 [==============================] - 159s 5s/step - loss: 6.8473 - masked_accuracy: 0.0719
Epoch 5/10
35/35 [==============================] - 151s 4s/step - loss: 6.8427 - masked_accuracy: 0.0712
Epoch 6/10
 6/35 [====>.........................] - ETA: 1:55 - loss: 6.7727 - masked_accuracy: 0.0743


KeyboardInterrupt



In [ ]:
# model.save('extra/transformer.keras')

In [ ]:
# model = keras.models.load_model('extra/transformer.keras')

In [42]:
def call(sentence, maxLength=256):
    tokenized = tokenizer.tokenize([sentence])
    sentence = tf.constant(tokenized)

    start = tf.cast(tf.constant(1)[tf.newaxis], tf.int64)
    end = tf.cast(tf.constant(2)[tf.newaxis], tf.int64)

    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in range(maxLength):
        output = tf.transpose(output_array.stack())
        predictions = model([sentence, output], training=False)
        
        predictions = predictions[:, -1:, :]
        predicted_id = tf.argmax(predictions, axis=-1)

        output_array = output_array.write(i+1, predicted_id[0])
        
        print(tokenizer.inverseMap[predicted_id.numpy()[0][0]])

        # if predicted_id == end:
        #     break

    output = tf.transpose(output_array.stack()).numpy()
    return output

In [43]:
from transformer import Translator
# translator = Translator(tokenizer, model)
output = call(input("Chat with Chicken: "))

Chat with Chicken:  hey man


<END>
<END>
<END>
<END>
<END>
<END>
<END>
<END>
<END>


KeyboardInterrupt: 

In [ ]:
model.summary(expand_nested=True)